In [1]:
!pip install h3

DEPRECATION: Loading egg at /srv/conda/envs/notebook/lib/python3.11/site-packages/odp-0.0.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Using cached h3-3.7.7-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.9 kB)
Using cached h3-3.7.7-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)


In [2]:
import pandas as pd
import h3
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import numpy as np

In [3]:
import pandas as pd
import h3
from tqdm import tqdm

In [4]:
input_file = 'output_occurence_data_classified/obis_classified.csv'
#output_file = f"output_H3binned_data/OBIS_h3counts_res{resolution}.csv"
output_file_base = 'output_H3binned_data/OBIS_h3counts'


In [24]:
test = pd.read_csv('output_occurence_data_classified/obis_classified.csv', nrows=10000000, dtype={'PrivateFlag': 'boolean', 'date_year': 'Int64'})

test

,dataset_id,decimalLongitude,decimalLatitude,date_year,PrivateFlag
0,0c1cb7e9-c7d1-4643-b245-daa8839a183f,-3.135050,58.911830,2009,False
1,0c1cb7e9-c7d1-4643-b245-daa8839a183f,-6.887870,58.217500,2010,False
2,0c1cb7e9-c7d1-4643-b245-daa8839a183f,1.281997,58.690365,2006,False
3,0c1cb7e9-c7d1-4643-b245-daa8839a183f,-1.104040,60.327460,2008,False
4,0c1cb7e9-c7d1-4643-b245-daa8839a183f,1.744569,61.393000,2006,False
...,...,...,...,...,...
9999995,5bac0fd0-82cd-47df-bb58-a7fe226194ac,-74.289810,10.988967,2020,False
9999996,2ae2a2bd-8412-405b-8a9f-b71adc41d4c5,-170.641190,-14.251471,2016,False
9999997,38535a43-2982-4479-88c1-8303f7ee8665,-12.181700,42.155000,1985,False
9999998,0ec403c9-23b3-4529-ae51-8a0877f22450,-64.934380,18.374250,2019,False


In [5]:
df_landoceanclass_level2 = pd.read_csv('../H3_LandOceanCoast_classification/h3_classification_level_2.csv')
df_landoceanclass_level4 = pd.read_csv('../H3_LandOceanCoast_classification/h3_classification_level_4.csv')
df_landoceanclass_level6 = pd.read_csv('../H3_LandOceanCoast_classification/h3_classification_level_6.csv')


In [6]:
df_landoceanclass = pd.concat([df_landoceanclass_level2, df_landoceanclass_level4, df_landoceanclass_level6])

In [9]:
# Define the resolutions you want to calculate
resolutions = [6, 4, 2]  # Start with the highest and work downwards

# Chunk size for processing the large file
chunksize = 10**6  # Adjust based on your memory capacity

In [10]:
# Initialize an empty DataFrame for the highest resolution
agg_hex_counts_highest = pd.DataFrame(columns=['h3_index', 'count', 'count_private'])

# Initialize a counter to track the number of processed rows
rows_processed = 0
#total_rows = sum(1 for _ in open(input_file)) - 1  # Adjust if there's a header
total_rows = 128621187

In [11]:
total_rows

128621187

In [12]:
# Process the file in chunks
for chunk in pd.read_csv(input_file, chunksize=chunksize, dtype={'PrivateFlag': 'boolean', 'date_year': 'Int64'}):
    # Drop rows with missing values in the specified columns
    chunk.dropna(subset=['decimalLatitude', 'decimalLongitude'], inplace=True)
    
    # Calculate H3 index for each row at the highest resolution (6)
    chunk['h3_index'] = chunk.apply(lambda row: h3.geo_to_h3(row['decimalLatitude'], row['decimalLongitude'], 6), axis=1)
    
    # Group by H3 index to count the number of points in each hexagon
    density = chunk.groupby('h3_index').agg(count=('h3_index', 'size'),
                                            count_private=('PrivateFlag', lambda x: (x == True).sum())).reset_index()
    
    # Aggregate this chunk's counts with the overall counts for the highest resolution
    agg_hex_counts_highest = pd.concat([agg_hex_counts_highest, density]).groupby('h3_index', as_index=False).sum()
    
    # Update the number of processed rows
    rows_processed += len(chunk)
    
    # Calculate progress percentage
    progress_percentage = (rows_processed / total_rows) * 100
    tqdm.write(f"Processed {rows_processed} rows ({progress_percentage:.2f}% complete)")

Processed 1000000 rows (0.78% complete)
Processed 2000000 rows (1.55% complete)
Processed 3000000 rows (2.33% complete)
Processed 4000000 rows (3.11% complete)
Processed 5000000 rows (3.89% complete)
Processed 6000000 rows (4.66% complete)
Processed 7000000 rows (5.44% complete)
Processed 8000000 rows (6.22% complete)
Processed 9000000 rows (7.00% complete)
Processed 10000000 rows (7.77% complete)
Processed 11000000 rows (8.55% complete)
Processed 12000000 rows (9.33% complete)
Processed 13000000 rows (10.11% complete)
Processed 14000000 rows (10.88% complete)
Processed 15000000 rows (11.66% complete)
Processed 16000000 rows (12.44% complete)
Processed 17000000 rows (13.22% complete)
Processed 18000000 rows (13.99% complete)
Processed 19000000 rows (14.77% complete)
Processed 20000000 rows (15.55% complete)
Processed 21000000 rows (16.33% complete)
Processed 22000000 rows (17.10% complete)
Processed 23000000 rows (17.88% complete)
Processed 24000000 rows (18.66% complete)
Processed 250

In [13]:
# Store results for each resolution
agg_hex_counts_by_res = {6: agg_hex_counts_highest}

# Aggregate to lower resolutions
for res in [4, 2]:
    agg_hex_counts_by_res[res] = agg_hex_counts_by_res[6].copy()
    agg_hex_counts_by_res[res]['h3_index'] = agg_hex_counts_by_res[6]['h3_index'].apply(lambda x: h3.h3_to_parent(x, res))
    agg_hex_counts_by_res[res] = agg_hex_counts_by_res[res].groupby('h3_index', as_index=False).sum()

# Merge each resolution's result with the classification data
for res in resolutions:
    agg_hex_counts_by_res[res] = pd.merge(agg_hex_counts_by_res[res], 
                                          df_landoceanclass[['h3_index', 'classification']], 
                                          on='h3_index', 
                                          how='left')

    # Calculate the centroid of each hexagon with 5 decimal places
    agg_hex_counts_by_res[res]['latitude'] = agg_hex_counts_by_res[res]['h3_index'].apply(lambda x: round(h3.h3_to_geo(x)[0], 5))
    agg_hex_counts_by_res[res]['longitude'] = agg_hex_counts_by_res[res]['h3_index'].apply(lambda x: round(h3.h3_to_geo(x)[1], 5))
    
    # Write the aggregated DataFrame to a CSV file
    output_file = f"{output_file_base}_res{res}.csv"
    agg_hex_counts_by_res[res].to_csv(output_file, index=False)
    
    print(f"Data successfully written to {output_file} for resolution {res}")

Data successfully written to output_H3binned_data/OBIS_h3counts_res6.csv for resolution 6
Data successfully written to output_H3binned_data/OBIS_h3counts_res4.csv for resolution 4
Data successfully written to output_H3binned_data/OBIS_h3counts_res2.csv for resolution 2


In [ ]:
# Define the resolutions you want to calculate
resolutions = [2, 4, 6]

# Chunk size for processing the large file
chunksize = 10**6  # Adjust based on your memory capacity


In [ ]:
# Iterate over each resolution
for resolution in resolutions:
    print(f"Processing resolution {resolution}...")
    
    # Initialize an empty DataFrame for aggregation
    agg_hex_counts = pd.DataFrame(columns=['h3_index', 'count', 'count_private'])
    
    # Initialize a counter to track the number of processed rows
    rows_processed = 0
    total_rows = sum(1 for _ in open(input_file)) - 1  # Adjust if there's a header
    
    # Process the file in chunks
    for chunk in pd.read_csv(input_file, chunksize=chunksize):
        # Drop rows with missing values in the specified columns
        chunk.dropna(subset=['decimalLatitude', 'decimalLongitude'], inplace=True)
        
        # Calculate H3 index for each row at the current resolution
        chunk['h3_index'] = chunk.apply(lambda row: h3.geo_to_h3(row['decimalLatitude'], row['decimalLongitude'], resolution), axis=1)
        
        # Group by H3 index to count the number of points in each hexagon
        density = chunk.groupby('h3_index').agg(count=('h3_index', 'size'),
                                                count_private=('PrivateFlag', lambda x: (x == True).sum())).reset_index()
        
        # Aggregate this chunk's counts with the overall counts
        agg_hex_counts = pd.concat([agg_hex_counts, density]).groupby('h3_index', as_index=False).sum()
        
        # Update the number of processed rows
        rows_processed += len(chunk)
        
        # Calculate progress percentage
        progress_percentage = (rows_processed / total_rows) * 100
        tqdm.write(f"Processed {rows_processed} rows ({progress_percentage:.2f}% complete)")

    # Merge the aggregated hex counts with the classification data
    agg_hex_counts = pd.merge(agg_hex_counts, df_landoceanclass[['h3_index', 'classification']], on='h3_index', how='left')
    
    # Calculate the centroid of each hexagon
    agg_hex_counts['latitude'] = agg_hex_counts['h3_index'].apply(lambda x: h3.h3_to_geo(x)[0])
    agg_hex_counts['longitude'] = agg_hex_counts['h3_index'].apply(lambda x: h3.h3_to_geo(x)[1])
    
    # Write the aggregated DataFrame to a CSV file
    output_file = f"{output_file_base}_res_{resolution}.csv"
    agg_hex_counts.to_csv(output_file, index=False)
    
    print(f"Data successfully written to {output_file} for resolution {resolution}")

In [2]:
# Set the resolution of H3 hexagons
resolution = 2

# Define chunk size
chunksize = 10**6  # Adjust this size according to your available memory

In [3]:
# File path
input_file = 'output_occurence_data_classified/obis_classified.csv'
output_file = f"output_H3binned_data/OBIS_h3counts_res{resolution}.csv"

total_rows = 128621187  # Approximate number of rows # NB PURELY FOR PROGRESS REPORTING, NOT ESSENTIAL

In [22]:
# File path
input_file = 'obis_classified_privateonly.csv'
output_file = f"output_H3binned_data/OBIS_h3counts_res{resolution}_privateonly.csv"

total_rows = 6440698  # Approximate number of rows # NB PURELY FOR PROGRESS REPORTING, NOT ESSENTIAL

In [4]:
output_file

'output_H3binned_data/OBIS_h3counts_res2.csv'

In [5]:
# Initialize an empty DataFrame for aggregation
agg_hex_counts = pd.DataFrame(columns=['h3_index', 'count'])

# Initialize a counter to track the number of processed rows
rows_processed = 0
# Process the file in chunks
for chunk in pd.read_csv(input_file, chunksize=chunksize):
    # Drop rows with missing values in the specified columns
    chunk.dropna(subset=['decimalLatitude', 'decimalLongitude'], inplace=True)
    
    # Convert latitude and longitude to H3 index
    chunk['h3_index'] = chunk.apply(lambda row: h3.geo_to_h3(row['decimalLatitude'], row['decimalLongitude'], resolution), axis=1)
    
    # Group by H3 index to count the number of points in each hexagon
    density = chunk['h3_index'].value_counts().reset_index()
    density.columns = ['h3_index', 'count']
    
    # Aggregate this chunk's counts with the overall counts
    agg_hex_counts = pd.concat([agg_hex_counts, density]).groupby('h3_index', as_index=False).sum()

    # Update the number of processed rows
    rows_processed += len(chunk)
    
    # Calculate progress percentage
    progress_percentage = (rows_processed / total_rows) * 100
    
    # Print progress message
    print(f"Processed {rows_processed} rows ({progress_percentage:.2f}% complete)")

/tmp/ipykernel_3995/143643203.py:7: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(input_file, chunksize=chunksize):


Processed 10000000 rows (7.77% complete)


/tmp/ipykernel_3995/143643203.py:7: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(input_file, chunksize=chunksize):


Processed 20000000 rows (15.55% complete)


/tmp/ipykernel_3995/143643203.py:7: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(input_file, chunksize=chunksize):


...

Processed 40000000 rows (31.10% complete)


/tmp/ipykernel_3995/143643203.py:7: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(input_file, chunksize=chunksize):


Processed 50000000 rows (38.87% complete)


/tmp/ipykernel_3995/143643203.py:7: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(input_file, chunksize=chunksize):


KeyboardInterrupt: 

In [ ]:
# Calculate the centroid of each hexagon
agg_hex_counts['latitude'] = agg_hex_counts['h3_index'].apply(lambda x: h3.h3_to_geo(x)[0])
agg_hex_counts['longitude'] = agg_hex_counts['h3_index'].apply(lambda x: h3.h3_to_geo(x)[1])

# Write the aggregated DataFrame to a CSV file
agg_hex_counts.to_csv(output_file, index=False)

print(f"Data successfully written to {output_file}")


In [29]:
df_hex = pd.read_csv(output_file)
df_hex

,h3_index,count,latitude,longitude
0,82000ffffffffff,2,81.957492,31.757228
1,820017fffffffff,1,78.128155,51.704538
2,82001ffffffffff,126,81.029276,50.504635
3,82002ffffffffff,1,79.649682,22.599927
4,820067fffffffff,22,80.953594,-6.231433
...,...,...,...,...
2724,82f377fffffffff,778,-72.680492,-151.356811
2725,82f387fffffffff,2,-80.953594,173.768567
2726,82f38ffffffffff,6,-81.905946,-168.639427
2727,82f3a7fffffffff,4589,-78.347546,174.341491
